# Randomisera deltagare till experiment (med do-fil)
English version | [Startsida](../index.html)

För att uttala oss om kausalitet skulle vi helst vilja jämföra varje observation med en kontrafaktisk version av sig själv, som är identisk i alla avseenden, men med ett annat värde på den oberoende variabeln. I medicinsk forskning kan man komma ganska nära det idealet, med klonade möss (till exempel). Då gör man något med en mus, och jämför det med en identisk mus som man inte gör någonting med. Det blir ungefär som att jämföra musen med sig själv, och ger goda möjligheter att dra slutsatser om orsak och verkan.

Men i många fall kan vi inte göra det. Vi kan då istället jämföra grupper, som är identiska på aggregerad nivå. Det vill säga, individerna i grupperna skiljer sig, men grupperna som helhet är lika. Sedan gör man något med den ena gruppen (experimentgruppen) och inte med den andra (kontrollgruppen).

**Det bästa sättet att skapa dessa två grupper är genom randomisering, det vill säga att vi slumpar in individer till grupperna.** Om det inte finns någon systematik i vem som hamnar var så kommer inte heller grupperna att skilja sig åt på något systematiskt sätt, givet att det är tillräckligt många i varje grupp.

Randomisering kan göras på olika sätt, till exempel genom att man slår en tärning. Ibland har vi dock en situation där vi vill skapa grupper som är lika stora. Vi kanske vill göra ett experiment med en kontrollgrupp och experimentgrupp, och vill ha 50 personer i varje grupp. Eller så vill vi ha tre grupper, med 100 personer i varje. I ett surveyexperiment handlar tilldelningen till kontrollgrupp och experimentgrupp om vilken version av enkäten man ska få.

[Här är en Stata do-fil som låter dig skapa en lista över vilka deltagare som ska ha olika versioner av en enkät, eller vilka experimentdeltagare som ska ingå i vilken grupp.](../data/dofil_randomisering.do)

Du behöver bara ställa in två saker: Antalet enkäter som ska delas ut (eller antalet deltagare) och hur många versioner du har av din enkät (eller hur många olika grupper du vill dela in dem i). Sedan kör man *hela* do-filen. Då får man ut en lista på antalet deltagare, och vilken version av enkäten/vilken grupp varje deltagare ska tillhöra. OBS! För att det ska fungera måste antalet deltagare vara jämnt delbart på antalet versioner/grupper. Alltså 50 deltagare och 2 grupper till exempel.

**Om du inte använder Stata så är här en Excelfil som gör samma sak.** Ange bara hur många deltagare du vill ha, och hur många olika versioner (eller grupper) du vill dela in dem i. Obs! Du behöver tillåta att Macron körs för att filen ska funka.

[Här är en Excelfil som låter dig skapa en lista över vilka deltagare som ska ha olika versioner av en enkät, eller vilka experimentdeltagare som ska ingå i vilken grupp.](../data/stathelp_randomisering.xls)

Båda filerna gör alltså samma sak. Nedan är en förklaring av principen i Stata-filen.

## En funktion för att randomisera

I det här avsnittet går jag igenom vad som står i do-filen för att få till randomiseringen, för den som är intresserad. Den som inte är intresserad kan bara ladda ned do-filen ovan och köra den.

Först rensar vi datan med `clear`, sedan ställer vi in de två värdena, "surveys" och "versions". Dessa värden sparas som globala [makron](macros_sv.html), som vi kan använda senare. Vi ställer in att vi vill ha 50 enkäter, och två versioner.

In [2]:
clear
global surveys = 50
global versions = 2

I nästa steg räknar vi ut hur många enkäter det ska vara per grupp. Det blir helt enkelt "surveys" delat med "versions": `global surveyspergroup = $surveys/$versions`. Sedan anger vi hur många observationer datasetet ska ha, och det är lika många som vi vill ha enkäter: `set obs $surveys`. Eftersom vi tidigare angett att "surveys" är 50 så blir det 50 observationer.

Slutligen gör vi en tom variabel som ska ange vilken version av enkäten varje person ska få. Variabeln får heta "version" `gen version = .`.

In [3]:
global surveyspergroup = $surveys/$versions

set obs $surveys
gen version = .



number of observations (_N) was 0, now 50

(50 missing values generated)


I nästa steg ska vi skapa en sekvens av versionsmarkörer. Först 25 ettor, och sedan 25 tvåor. Vi kommer sedan lägga dem i slumpmässig ordning. Vi gör detta med en dubbelloop ([läs mer om loopar här](forvalues_sv.html)).

Men först gör vi ett globalt makro som anger vilken observation vi ska hålla på med: `global obsnr = 1`. Vi kommer använda det inne i looparna.

Sedan använder vi `forvalues` för att loopa igenom antalet versioner av enkäten `forvalues vnr = 1/$versions {`.

Inne i den loopen gör vi en till loop, där vi loopar över antalet enkäter i varje grupp `forvalues snr = 1/$surveyspergroup {`.

Den första loopen kör alltså i det här fallet 2 gånger, och den andra loopen kör 25 gånger. Inne i den andra loopen gör två saker: sätter ut en etta eller tvåa i den aktuella observationen:, och sedan plussar vi på ett i vårt makro "obsnr", så att loopen nästa vända sätter ut ettan eller tvåan i nästa observation: `global obsnr = $obsnr+1`



In [6]:
global obsnr = 1
quietly forvalues vnr = 1/$versions {
 forvalues snr = 1/$surveyspergroup {
  replace version = `vnr' in $obsnr
  global obsnr = $obsnr+1
 }
}

Om man nu tittar i datan kommer vi se en variabel "version", med 25 ettor och sedan 25 tvåor. Nu behöver vi bara lägga dem i en slumpmässig ordning.

Vi gör det genom att först skapa en ny variabel som består av slumptal, mellan 0 och 1. Varje observation får alltså sitt eget slumptal. Vi gör detta genom `gen randomnr = runiform()`. Den nya variabeln heter alltså "randomnr". Sedan sorterar vi den i storleksordning med `sort randomnr`, och så tar vi sedan bort den nu onödiga slumpvariabeln med `drop randomnr`.

Nu har vi kastat om listan. Vi har fortfarande 25 ettor och 25 tvåor, men nu ligger de i slumpmässig ordning, precis som vi var ute efter.

In [7]:
gen randomnr = runiform()
sort randomnr
drop randomnr

Slutligen gör vi en ny variabel som bara är ordningen, alltså 1, 2, 3, 4, 5 osv. Det gör vi med `egen surveyorder = fill(1 2 3)`. Vi anger mönstret i "fill", så nu räknar den upp från 1 2 3.

Sedan byter vi plats på variablerna (inte nödvändigt men ser prydligare ut) med `move surveyorder version`. Slutligen kan vi ta ut en lista på de första 10 raderna med `list surveyorder version in 1/10, clean`.

In [8]:
egen surveyorder = fill(1 2 3)
move surveyorder version
list surveyorder version in 1/10, clean





       survey~r   version  
  1.          1         1  
  2.          2         2  
  3.          3         2  
  4.          4         1  
  5.          5         1  
  6.          6         1  
  7.          7         2  
  8.          8         1  
  9.          9         2  
 10.         10         1  


Klart! Person 1 ska ha version 1 av enkäten, person 2 ska ha version 2, person 3 ska ha version 2, och så vidare.

Det finns givetvis flera olika sätt att uppnå det här resultatet, det här var bara ett. Och det viktiga är inte hur det går till, utan bara att det blir ordentligt slumpat, så att det inte blir någon systematik i vem som hamnar i vilken grupp eller vem som får vilken enkät.